In [167]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_account.ipynb 

In [168]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a blocks summary. It only stores the `root` node of the txs merkle tree, which makes it small and constant in size. The actual mining happens on the block header.

In [169]:
class Header:
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        super().__setattr__('hash', sha(self.__dict__))
    
    def __str__(self):       return stringfy(self)
    def __eq__(self, other): return self.hash == other.hash

In [170]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          📗 0x710ce47086...9b8
hash:          👽 0x174ba26c87...2da
prev_hash:     💿 0x594d25d702...8ef
number:        2
n_txs:         10
mined:         False
time:          Tue Mar 30 20:36:05 2021


If anything in the block header changes its `hash` changes automatically as well.

In [171]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

Calculate the total volume and fees included in a block.

In [172]:
def stats(txs):
    volume = sum([tx.value for tx in txs])
    fees   = sum([tx.fee   for tx in txs])
    return volume, fees

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [221]:
class Block: 
    def __init__(self, bh, txs):
        bh.volume,bh.fees = stats(txs)
        self.bh  = bh
        self.mt  = MerkleTree(txs)
        self.txs = self.val_txs(txs)
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),   'tx signature invalid'
        assert self.are_unique(txs),         'txs include duplicates'
        assert self.mt.root == self.bh.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def __getitem__(self, key):return self.txs[key]
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __eq__(self, other):   return super().__eq__()
    def __str__(self):         return (str(self.bh) 
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(ph(tx) for tx in self.txs.keys()))

Create random list of signed txs.

In [222]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [223]:
txs = r_stxs(2)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          📌 0x667bfcd0b0...3fa
hash:          💫 0x452cc08b0e...b09
prev_hash:     🕁 0xdbe55ad234...651
number:        2
n_txs:         2
mined:         False
time:          Tue Mar 30 20:48:31 2021
volume:        7.0 ether
fees:          0.05 ether

txs:
📷 0x91ec35021f...86d
📊 0x64f0e397a5...566


Access specific tx from block with its hash.

In [227]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0x91ec35021f53ccef54e11c8357461e961bc9713f8d2bcbaa4ff6e7e1d7e9786d'

In [228]:
tx = b[tx_hash]; print(tx)

fr:            🔚 0xB46725bb73...911
hash:          📷 0x91ec35021f...86d
to:            👶 0x10fbD4FfBa...f25
value:         5.0 ether
fee:           0.04 ether
nonce:         0
time:          Tue Mar 30 20:48:31 2021
signed:        True
sig:           📱 0x8b6bcf1187...2a7


Blocks can only contain unique txs. 

In [229]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [230]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected